In [16]:
from utils.model_utils import *
from utils.bitassign_utils import *
from nats_bench import create

api = create('/home/dell/dataset/NATS-tss-v1_0-3ffb9-full_cat/NATS-tss-v1_0-3ffb9-full', 'tss', fast_mode=True, verbose=False)
train_loader, valid_loader, test_loader = load_data('cifar10', '~/dataset')

In [20]:
from utils.model_utils import *
from utils.bitassign_utils import *

from nats_bench import create

# api = create('/home/dell/dataset/NATS-tss-v1_0-3ffb9-full_cat/NATS-tss-v1_0-3ffb9-full', 'tss', fast_mode=True, verbose=False)
model_path = '/home/dell/MP-NAS-Bench201/results/models/test_50_quant_cifar10_model_6595_1.pth'
filename = '/home/dell/MP-NAS-Bench201/config/6595_1.yaml'

H1 = {'epochs': 50, 'lr': 0.01, 'batch_size': 256}

# model = get_network(api, 490, quant = True)

config = api.get_net_config(6595, 'cifar10')
model = get_cell_based_tiny_net(config)

# Load the pre-trained weights: params is a dict, where the key is the seed and value is the weights.
# params = api.get_net_param(490, 'cifar10', None, hp = 200)

# set_quantization_bits_infercell(model, 3, w_bit = 4, a_bit= 8 )
# set_quantization_bits_infercell(model, 4, w_bit = 2, a_bit= 8 )
convert_conv2d_to_qconv2d(model, w_bit=8, a_bit=8 )
convert_linear_to_qlinear(model, w_bit=8, a_bit=8)

apply_bit(model,filename)

print(model)
acc = test_model(model, test_loader, len(test_loader)*H1['batch_size'])

model.load_state_dict(torch.load(model_path))

acc = test_model(model, test_loader, len(test_loader)*H1['batch_size'])

TinyNetwork(
  TinyNetwork(C=16, N=5, L=17)
  (stem): Sequential(
    (0): QConv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False, w_bit=8, a_bit=8, half wave)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (cells): ModuleList(
    (0-4): 5 x InferCell(
      info :: nodes=4, inC=16, outC=16, [1<-(I0-L0) | 2<-(I0-L1,I1-L2) | 3<-(I0-L3,I1-L4,I2-L5)], |nor_conv_1x1~0|+|avg_pool_3x3~0|avg_pool_3x3~1|+|nor_conv_1x1~0|nor_conv_3x3~1|avg_pool_3x3~2|
      (layers): ModuleList(
        (0): ReLUConvBN(
          (op): Sequential(
            (0): ReLU()
            (1): QConv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False, w_bit=4, a_bit=8, half wave)
            (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
        )
        (1-2): 2 x POOLING(
          (op): AvgPool2d(kernel_size=3, stride=1, padding=1)
        )
        (3): ReLUConvBN(
          (op): Sequenti

In [12]:


def apply_bit(model, filename):
    with open(filename, 'r') as file:
        bitwidth_table = yaml.safe_load(file)
    for name, child in model.named_children():
        if name == 'stem':
            child[0].w_bit = bitwidth_table['stem']['qconv']['w_bit']
            child[0].a_bit = bitwidth_table['stem']['qconv']['a_bit']
        if name =='cells':
            infer_list = [i for r in (range(0, 5), range(6, 11), range(12, 17)) for i in r]
            for j in infer_list:
                for i in bitwidth_table['infercell'].keys():
                    set_quantization_bits_infercell(child[j], i, w_bit=bitwidth_table['infercell'][i]['w_bit'], a_bit=bitwidth_table['infercell'][i]['a_bit'])
            
            for keys in bitwidth_table['resblock'].keys():
                set_quantization_bits_blockcell(child[5], keys, w_bit=bitwidth_table['resblock'][keys]['w_bit'], a_bit=bitwidth_table['resblock'][keys]['a_bit'])
                set_quantization_bits_blockcell(child[11], keys, w_bit=bitwidth_table['resblock'][keys]['w_bit'], a_bit=bitwidth_table['resblock'][keys]['a_bit'])
        if name =='classifier': 
            child.w_bit = bitwidth_table['classifier']['qlinear']['w_bit']
            child.a_bit = bitwidth_table['classifier']['qlinear']['a_bit']
    # print('Set bit width: {}'.format(self.bitwidth_table))